In [1]:
import tensorflow as tf
import scipy.io as scp
import numpy as np

In [2]:
testData = scp.loadmat('../data/svhn/test_32x32.mat')
trainData = scp.loadmat('../data/svhn/train_32x32.mat')

In [3]:
testDataX = testData['X']
testDataY = testData['y']

trainDataX = trainData['X']
trainDataY = trainData['y']

In [4]:
print type(trainDataX)
print type(trainDataY)

print trainDataX.shape
print trainDataY.shape
print testDataX.shape
print testDataY.shape

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(32, 32, 3, 73257)
(73257, 1)
(32, 32, 3, 26032)
(26032, 1)


In [5]:
# try tansposing the array
def transposeArray(data):
    print 'started'
    xtrain = []
    trainLen = data.shape[3]
    print trainLen
    for x in xrange(trainLen):
        xtrain.append(data[:,:,:,x])
    
    xtrain = np.asarray(xtrain)
    return xtrain

In [6]:
trainDataX = transposeArray(trainDataX)
testDataX = transposeArray(testDataX)


print trainDataX.shape

started
73257
started
26032
(73257, 32, 32, 3)


In [7]:
def OnehotEndoding(Y):
    Ytr=[]
    for el in Y:
        temp=np.zeros(10)
        if el==10:
            temp[0]=1
        elif el==1:
            temp[1]=1
        elif el==2:
            temp[2]=1
        elif el==3:
            temp[3]=1
        elif el==4:
            temp[4]=1
        elif temp[5]==1:
            temp[5]=1
        elif temp[6]==1:
            temp[6]=1
        elif temp[7]==1:
            temp[7]=1
        elif temp[8]==1:
            temp[8]=1
        elif temp[9]==1:
            temp[9]=1
        Ytr.append(temp)
        
    return np.asarray(Ytr)

In [8]:
# convert y to one hot encoding
trainDataY = OnehotEndoding(trainDataY)
testDataY = OnehotEndoding(testDataY)
print trainDataY.shape
print testDataY.shape

(73257, 10)
(26032, 10)


In [9]:
height = 32
width = 32
channel = 3
n_classes = 10
learning_rate = 0.01
hidden = 100

In [10]:
def WeightBias(input, output):
    w = tf.Variable(tf.random_normal([input, output]))
    b = tf.Variable(tf.random_normal([output]))
    return w, b

In [11]:
#for fully connected we will reshape it to a flat one
trainDataX = trainDataX.reshape((-1, height * width * channel))
testDataX = testDataX.reshape((-1, height * width * channel))
print trainDataX.shape


x = tf.placeholder(tf.float32, shape=(None, height * width * channel))
y = tf.placeholder(tf.float32, shape=(None, n_classes))

w1, b1 = WeightBias(height * width * channel, hidden)
w2, b2 = WeightBias(hidden, n_classes)

(73257, 3072)


In [12]:
def model(X, w1, b1, w2, b2):
    h1 = tf.nn.relu(tf.matmul(X, w1) + b1)
    return tf.matmul(h1, w2) + b2

In [13]:
pred = model(x, w1, b1, w2, b2)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [14]:
# Initializing the variables
init = tf.initialize_all_variables()
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(y,1)), "float"))

In [15]:
num_steps = 50
batch = 100

with tf.Session() as sess:
    sess.run(init)
    print('Initialized')
    
    for step in range(num_steps):
        p = np.random.permutation(range(len(trainDataX)))
        trX, trY = trainDataX[p], trainDataY[p]
        
        for start in range(0, len(trainDataX), batch):
            end = start + batch
            sess.run(optimizer, feed_dict={x:trX[start:end], y:trY[start:end]})
        
        print 'cost at step : ', step, sess.run(loss, feed_dict={x:trX, y:trY})

    
    # calculate the accuracy on the training data
    print 'accuracy on training data', sess.run(accuracy, feed_dict={x:trainDataX, y:trainDataY})
    
    print "Accuracy on test data:", accuracy.eval({x: testDataX, y: testDataY})

Initialized
cost at step :  0 1.2839
cost at step :  1 1.17909
cost at step :  2 1.13914
cost at step :  3 1.12046
cost at step :  4 1.11034
cost at step :  5 1.10432
cost at step :  6 1.10078
cost at step :  7 1.09836
cost at step :  8 1.09728
cost at step :  9 1.09657
cost at step :  10 1.09603
cost at step :  11 1.0955
cost at step :  12 1.09538
cost at step :  13 1.09533
cost at step :  14 1.0952
cost at step :  15 1.09514
cost at step :  16 1.09501
cost at step :  17 1.095
cost at step :  18 1.095
cost at step :  19 1.09491
cost at step :  20 1.09495
cost at step :  21 1.09481
cost at step :  22 1.09504
cost at step :  23 1.09508
cost at step :  24 1.0951
cost at step :  25 1.09499
cost at step :  26 1.09503
cost at step :  27 1.09485
cost at step :  28 1.09486
cost at step :  29 1.09496
cost at step :  30 1.09494
cost at step :  31 1.09506
cost at step :  32 1.09503
cost at step :  33 1.09499
cost at step :  34 1.09494
cost at step :  35 1.09479
cost at step :  36 1.09503
cost at